In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
import re

# Set up the Selenium driver
driver = webdriver.Edge(executable_path=r'path_to_your_webdriver')

# Step 1: Log in
login_url = 'your_login_url'
driver.get(login_url)
username_field = driver.find_element(By.NAME, 'username_field')
password_field = driver.find_element(By.NAME, 'password_field')
username_field.send_keys('your_username')
password_field.send_keys('your_password')
password_field.submit()  # submit the form
time.sleep(5)  # wait for the page to load

# Step 2: Go to the target page
target_page_url = 'your_target_page_url'
driver.get(target_page_url)
time.sleep(5)  # wait for the page to load

# Parse the page source with BeautifulSoup
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

# Find all 'td' elements that contain specific names (update regex with desired patterns)
name_tds = soup.find_all(
    'td',
    text=re.compile(
        r'Name1|Name2|Name3|...'  # Add all needed names here
    )
)

# Create a DataFrame to store all data
df = pd.DataFrame(columns=['Column1', 'Column2', 'Column3', 'Column4'])  # Customize column names as needed

# Iterate over each 'name_td'
for name_td in name_tds:
    # If we found such a tag, we get the next 'td' tag
    if name_td:
        next_td = name_td.find_next_sibling('td')

        # If the next 'td' tag exists, find the 'a' tag within it and get its 'href'
        if next_td:
            a = next_td.find('a', href=True)
            if a:
                target_link = a['href']

                # Construct the full URL
                full_url = 'your_base_url' + target_link

                # Navigate to the target link
                driver.get(full_url)
                time.sleep(5)

                # Parse the page source with BeautifulSoup
                page_source = driver.page_source
                soup = BeautifulSoup(page_source, 'html.parser')

                # Scrape the required data for each row
                # Find tbody first
                tbody = soup.find('tbody')

                data = {'Column1': [], 'Column2': [], 'Column3': [], 'Column4': []}  # Adjust as needed

                if tbody:
                    rows = tbody.find_all('tr')
                    for row in rows:
                        # Example logic to filter and extract data
                        first_td = row.find('td')
                        if first_td and first_td.text.strip() == 'ExcludeCondition':
                            continue
                        if row.find('form') is None or row.find('input') is None:
                            continue

                        # Change the indices to match actual column positions
                        column1_data = row.find_all('td')[0].text if row.find_all('td') else None
                        column2_data = row.find('a', target='newtop')['href'] if row.find('a', target='newtop') else None
                        column3_data = row.find_all('td')[2].text if len(row.find_all('td')) > 2 else None

                        # Store the data
                        data['Column1'].append(name_td.text)
                        data['Column2'].append(column1_data)
                        data['Column3'].append(column2_data)
                        data['Column4'].append(column3_data)

                    # Convert the data to a DataFrame
                    df = df.append(pd.DataFrame(data), ignore_index=True)

driver.quit()  # close the browser when done

# Process the 'Column3' column if necessary (Example provided)
# df[['Discard', 'Column3']] = df['Column3'].str.split('some_pattern=', expand=True)
# df = df.drop(columns=['Discard'])

# Example: Filter out already processed items
try:
    processed_ids_df = pd.read_excel('path_to_existing_file.xlsx')
    processed_ids = processed_ids_df['identifier_column'].tolist()
except FileNotFoundError:
    processed_ids = []

df = df[~df['Column3'].isin(processed_ids)]

# Example: Count occurrences of specific names in 'Column4'
names = ['Name1', 'Name2', 'Name3', ... ]
counts = {name: df['Column4'].str.contains(name, case=False).sum() for name in names}
counts_df = pd.DataFrame.from_dict(counts, orient='index', columns=['Count'])
counts_df.index.name = 'Identifier'

# Example: Common errors in 'Column4' using n-grams
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(2, 6), max_features=1000)
word_vectors = vectorizer.fit_transform(df['Column4'])
words = vectorizer.get_feature_names()
counts = word_vectors.sum(axis=0).A1

common_errors_df = pd.DataFrame({'Common Error': words, 'Counts': counts})
for name in names:
    common_errors_df = common_errors_df[~common_errors_df['Common Error'].str.contains(name, case=False)]
common_errors_df = common_errors_df.nlargest(100, 'Counts')

# Export the DataFrame and analysis to an Excel file
output_path = 'output_file.xlsx'
with pd.ExcelWriter(output_path) as writer:
    df.to_excel(writer, sheet_name='Scraped Data', index=False)
    counts_df.reset_index().to_excel(writer, sheet_name='Counts', index=False)
    common_errors_df.to_excel(writer, sheet_name='Common Errors', index=False)

# Example: Count occurrences of key phrases
key_phrases = ['key_phrase_1', 'key_phrase_2', ...]
key_phrase_counts = {phrase: df['Column4'].str.contains(phrase, case=False, regex=True).sum() for phrase in key_phrases}
key_phrase_counts_df = pd.DataFrame.from_dict(key_phrase_counts, orient='index', columns=['Count'])
key_phrase_counts_df.index.name = 'Key Phrase'

# Save the updated processed IDs
processed_ids.extend(df['Column3'].tolist())
processed_ids_df = pd.DataFrame(processed_ids, columns=['identifier_column'])
processed_ids_df.to_excel('path_to_existing_file.xlsx', index=False)

# Save the key phrase counts DataFrame to the same Excel file
with pd.ExcelWriter(output_path, engine='openpyxl', mode='a') as writer:
    key_phrase_counts_df.reset_index().to_excel(writer, sheet_name='Key Phrase Counts', index=False)